In [4]:
import glob
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict

In [5]:
dataset = 'ontonotes'
supervised_results_test = {
    'conll2003': [88.91970295455273, 0.6499490802070589],
    'ontonotes': [86.11677066546363, 0.38365896572169766],
}
mean, std = supervised_results_test[dataset]



files = glob.glob(f'/home/rvacareanu/projects_5_22/odinsynth_al/results/230103/{dataset}/entity*.jsonl') + glob.glob(f'/home/rvacareanu/projects_5_22/odinsynth_al/results/230105/{dataset}/entity*.jsonl')
data = []
for f in [x for x in files]:
    with open(f) as fin:
        for line in fin:
            data.append(json.loads(line))
# display(df)

groupby_parameters = ['active_learning_iteration', 'annotation_strategy', 'query_strategy_function', 'number_of_al_iterations', 'starting_size_ratio', 'underlying_model', 'epochs', 'number_of_new_examples', 'learning_rate', 'initial_dataset_selection_strategy', 'training_annotation_style', 'dataset_name', 'use_postags_for_selection']
unrolled_data = []
dd1 = defaultdict(int)
dd2 = defaultdict(int)
for x in data:
    for l in x:
        key = tuple([l[x] for x in groupby_parameters])
        dd1[key] += l['number_of_annotated_tokens']
        dd2[key] += 1
for x in data:
    for l in x:
        key = tuple([l[x] for x in groupby_parameters])
        unrolled_data.append(
            {
                **{k:v for (k, v) in l.items() if k not in ['selected_data_distribution', 'all_args']},
                'f1'                                     : 100 * l['test_metrics']['test_overall_f1'],
                'number_of_annotated_tokens_smoothed'    : dd1[key]/dd2[key],
                'percentage_of_annotated_tokens_smoothed': 100 * (dd1[key]/dd2[key]) / l['total_number_of_tokens_available'],
            }
        )

data = unrolled_data


In [6]:

df = pd.DataFrame(data)


with pd.option_context('display.max_colwidth', None, 'display.max_columns', None, 'display.max_rows', None):
    df = df[((df['query_strategy_function'] == 'breaking_ties_query'))]
    df['name'] = ''
    df.loc[((df['query_strategy_function'] == 'breaking_ties_query') & (df['initial_dataset_selection_strategy'] == 'random_initial_dataset_sampling')), 'name']                      = 'Random'
    df.loc[((df['query_strategy_function'] == 'breaking_ties_query') & (df['initial_dataset_selection_strategy'] == 'nnp_filter_initial_dataset_sampling')), 'name']                  = 'NNP Filter'
    df.loc[((df['query_strategy_function'] == 'breaking_ties_query') & (df['initial_dataset_selection_strategy'] == 'models_breaking_ties_lm_predictions_dataset_sampling')), 'name'] = 'LM prediction guided'
    df.rename(columns={'name': 'Initial Dataset Selection'}, inplace=True)

    print(df.groupby(by=['Initial Dataset Selection', 'percentage_of_annotated_tokens_smoothed']).agg({'f1': ['mean', 'std']}).unstack(level='Initial Dataset Selection'))
    a()



# print(df['Annotation Level'].unique())
df = df[((df['query_strategy_function'] == 'breaking_ties_query'))]
df['name'] = ''
df.loc[((df['query_strategy_function'] == 'breaking_ties_query') & (df['initial_dataset_selection_strategy'] == 'random_initial_dataset_sampling')), 'name']                      = 'Random'
df.loc[((df['query_strategy_function'] == 'breaking_ties_query') & (df['initial_dataset_selection_strategy'] == 'nnp_filter_initial_dataset_sampling')), 'name']                  = 'NNP Filter'
df.loc[((df['query_strategy_function'] == 'breaking_ties_query') & (df['initial_dataset_selection_strategy'] == 'models_breaking_ties_lm_predictions_dataset_sampling')), 'name'] = 'LM prediction guided'
df.rename(columns={'name': 'Initial Dataset Selection'}, inplace=True)
# display(df[df['active_learning_iteration'] == 0])
# a()
# display(df[((df['active_learning_iteration']==0) & (df['seed']==1))][['active_learning_iteration', 'seed', 'annotation_strategy', 'query_strategy_function', 'f1', 'percentage_of_annotated_tokens_smoothed']])
# a()
# sns.lineplot(data=df[df['training_annotation_style'] == 'mask_all_unknown'], x='active_learning_iteration', y='f1', hue='Query Strategy', errorbar='sd', palette=['#0173b2', '#de8f05', '#029e73', '#d55e00'])
# plt.figure(figsize=(12,8))
sns.color_palette("colorblind")
g = sns.lineplot(data=df, x='percentage_of_annotated_tokens_smoothed', y='f1', hue='Initial Dataset Selection', hue_order=['Random', '', 'LM prediction guided'], errorbar='sd')
g.set_xticks([0, 10, 20, 30, 40, 50, 60, 70, 80])
plt.axhline(mean, 0, 1, label='Supervised', color='#fbafe4', linestyle='--')

# g = sns.lineplot(data=df, x='percentage_of_annotated_tokens_smoothed', y='f1', hue='Initial Dataset Selection', hue_order=['Random', 'LM prediction guided'], errorbar='sd')


plt.xlabel("Percentage of the total number of tokens annotated", fontsize=14)
plt.ylabel('F1', fontsize=14)
plt.legend(loc='lower right', fontsize=16)
plt.tight_layout()

                                                          f1             \
                                                        mean              
Initial Dataset Selection               LM prediction guided NNP Filter   
percentage_of_annotated_tokens_smoothed                                   
0.972751                                                 NaN        NaN   
1.506718                                           74.124294        NaN   
1.747292                                                 NaN  74.970395   
2.829696                                                 NaN        NaN   
3.265695                                           81.080111        NaN   
3.406697                                                 NaN  79.965014   
4.237154                                                 NaN        NaN   
4.677167                                           83.748413        NaN   
4.819243                                                 NaN  84.024474   
5.604068                 

NameError: name 'a' is not defined

In [ ]:
print(sns.color_palette('colorblind').as_hex())

['#0173b2', '#de8f05', '#029e73', '#d55e00', '#cc78bc', '#ca9161', '#fbafe4', '#949494', '#ece133', '#56b4e9']
